In [11]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from collections import Counter 
from sklearn.metrics import accuracy_score , r2_score , f1_score, classification_report, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor  
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

In [3]:
data = pd.read_excel('H:\DATA\MY\practice\Practice-22\Raisin_Dataset.xlsx')
data.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [9]:
print(data.shape)
print(Counter(data['Class']))
data = data.round(2)

(900, 8)
Counter({'Kecimen': 450, 'Besni': 450})


In [14]:
# we are performing first , LinearRegression , the target variable is AREA 

from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()
data['Class'] = encode.fit_transform(data['Class'])


X = data.drop('Area',axis = 1)
Y = data['Area']

x_train, x_test, y_train, y_test = train_test_split(X,Y , test_size=0.2 , random_state= 42)

model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

r2 = r2_score(y_pred , y_test)
mse = mean_squared_error(y_pred, y_test)

print('the R2 score : ',r2)
print('the mean squared error :',mse)

the R2 score :  0.9963475974401299
the mean squared error : 5306226.943547243


In [16]:
from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(model, X, Y, cv=5, scoring='r2')

# Output the accuracy for each fold
print("Cross-Validation r2 Scores: ", cv_results)

# Output the mean and standard deviation of the accuracy scores
print("Mean Accuracy: ", cv_results.mean())
print("Standard Deviation: ", cv_results.std())



Cross-Validation r2 Scores:  [0.99508929 0.99363772 0.99549037 0.99673883 0.9962876 ]
Mean Accuracy:  0.9954487622249604
Standard Deviation:  0.0010750596271208476


In [19]:
X = data.drop('Class', axis = 1)
Y = data['Class']

model = RandomForestClassifier()

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state= 42)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

accuracy = accuracy_score(y_pred, y_test)
repport = classification_report( y_pred, y_test)

print('the accuracy score : ',accuracy)
print(repport)

the accuracy score :  0.85
              precision    recall  f1-score   support

           0       0.84      0.85      0.84        85
           1       0.86      0.85      0.86        95

    accuracy                           0.85       180
   macro avg       0.85      0.85      0.85       180
weighted avg       0.85      0.85      0.85       180



In [21]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Assuming X and Y are already defined
X = data.drop('Class', axis=1)
Y = data['Class']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],     # Number of boosting rounds
    'max_depth': [3, 5, 7],             # Maximum depth of a tree
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage
    'subsample': [0.6, 0.8, 1.0],       # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0] # Subsample ratio of columns when constructing each tree
}

# Initialize the XGBoost Classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit the model
grid_search.fit(x_train, y_train)

# Best parameters found by GridSearchCV
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Best model
best_model = grid_search.best_estimator_

# Make predictions with the best model
y_pred = best_model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print('The accuracy score after tuning:', accuracy)
print('Classification Report after tuning:\n', report)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
The accuracy score after tuning: 0.8722222222222222
Classification Report after tuning:
               precision    recall  f1-score   support

           0       0.86      0.87      0.87        86
           1       0.88      0.87      0.88        94

    accuracy                           0.87       180
   macro avg       0.87      0.87      0.87       180
weighted avg       0.87      0.87      0.87       180



c:\Users\mouncef\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
